<a href="https://colab.research.google.com/github/davidwhogg/rvSensitivity/blob/master/ipynb/toy_rv_sensitivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toy experiments in RV sensitivity

Just checking the basics.

In [0]:
import numpy as np
import pylab as plt
#matplotlib inline

In [0]:
def make_observing_program(N, T, periodic=False):
  """
  really dumb observing plans
  """
  if periodic:
    dT = T / N
    return np.arange(0.5 * dT, T, dT)
  return np.sort(T * np.random.uniform(size=N))

In [0]:
Ny = 5 # 5-year survey
N = 150 * Ny
T = 365.25 * Ny
ts = make_observing_program(N, T)
ts.shape

In [0]:
# mildly heteroskedastic data around 30 cm/s
sigma = 30.
sigmas = sigma - 0.1 * sigma + 0.2 * sigma * np.random.uniform(size=N) # cm/s
ivars = 1. / sigmas ** 2

In [0]:
# logarithmic period grid
Pmin = 2. ** 4 # days
Pmax = 2. ** 14 # days
Ps = np.exp(np.arange(np.log(Pmin), np.log(Pmax)+0.001, 0.125))
Ps.shape

In [0]:
def design_matrix(P, phase, ts):
  """
  This is supposed to do the Kepler problem but it doesn't!
  """
  return np.vstack((np.cos(2. * np.pi * ts / P + phase), np.ones_like(ts))).T

def precisions(Ps, ts, ivars):
  # coarse grid for phase integral
  Nphase = 16
  dph = 2. * np.pi / Nphase
  phases = np.arange(0.5 * dph, 2. * np.pi, dph)
  infos = np.zeros_like(Ps)
  for ii, P in enumerate(Ps):
    for phi in phases:
      A = design_matrix(P, phi, ts)
      infos[ii] += (1. / np.linalg.inv(A.T * ivars @ A)[0, 0]) / Nphase # doing an integral
  return infos

In [0]:
infos_P = precisions(Ps, ts, ivars)
uncertainties_P = 1. / np.sqrt(infos_P)

In [0]:
plt.axvline(T)
plt.axhline(sigma / np.sqrt(N / 2))
plt.plot(Ps, uncertainties_P, "k.")
plt.semilogx()
plt.ylim(0., 10.)
plt.ylabel("expected uncertainty on kappa (cm/s)")
plt.xlabel("orbital period P (days)")
plt.title("sensitivity of a {:d}-year survey as a function of period P".format(Ny))

In [0]:
fiducialP = 300.0 # days
Nmin = (fiducialP / T) ** 2 * N
Ns = np.ceil(np.exp(np.linspace(np.log(Nmin), np.log(N), 32))).astype(int)
Nsubsurvey = len(Ns)
Ts = np.zeros(Nsubsurvey)
infos_T = np.zeros(Nsubsurvey)
for subsurvey in range(Nsubsurvey):
  tsthis = ts[:Ns[subsurvey]]
  ivarsthis = ivars[:Ns[subsurvey]]
  Ts[subsurvey] = np.max(tsthis) - np.min(tsthis) # dumb
  infos_T[subsurvey] = precisions([fiducialP, ], tsthis, ivarsthis)
uncertainties_T = 1. / np.sqrt(infos_T)

In [0]:
plt.axvline(fiducialP)
plt.plot(Ts, sigma / np.sqrt((Ns / 2)))
plt.plot(Ts, uncertainties_T, "k.")
plt.ylabel("expected uncertainty at P = 300 d (cm/s)")
plt.xlabel("survey duration T (days)")
plt.title("sensitivity to a {:3.0f}-d planet as a function of survey duration".format(fiducialP))
plt.loglog()